In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob # for combining csvs

In [96]:
data_files = glob.glob('./data/*.csv')

The data is presented in seperate csv files for each year. I already started the filtering process by only grabbing data from 2000 onwards.  This is because I only want to look at more modern tennis.

In [97]:
df = pd.concat(
  (pd.read_csv(filename) for filename in data_files), 
  ignore_index=True
)

df
df_raw = df

## Preprocessing and EDA

For this section I will look to process and filter the data, and then look to perform some EDA on it.

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73247 entries, 0 to 73246
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          73247 non-null  object 
 1   tourney_name        73247 non-null  object 
 2   surface             73194 non-null  object 
 3   draw_size           73247 non-null  int64  
 4   tourney_level       73247 non-null  object 
 5   tourney_date        73247 non-null  int64  
 6   match_num           73247 non-null  int64  
 7   winner_id           73247 non-null  int64  
 8   winner_seed         30444 non-null  object 
 9   winner_entry        9229 non-null   object 
 10  winner_name         73247 non-null  object 
 11  winner_hand         73240 non-null  object 
 12  winner_ht           71351 non-null  float64
 13  winner_ioc          73247 non-null  object 
 14  winner_age          73240 non-null  float64
 15  loser_id            73247 non-null  int64  
 16  lose

In [99]:
df = df.sort_values(by='tourney_date')
df

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
67565,2000-339,Adelaide,Hard,32,A,20000103,7,102872,NaN,NaN,James Sekulov,R,180.0,AUS,23.2,101150,NaN,NaN,Gianluca Pozzi,L,180.0,ITA,34.5,6-4 2-6 6-2,3,R32,89.0,2.0,3.0,74.0,50.0,32.0,14.0,13.0,3.0,6.0,8.0,5.0,64.0,26.0,22.0,17.0,13.0,2.0,6.0,144.0,325.0,94.0,502.0
69919,2000-891,Chennai,Hard,32,A,20000103,4,101086,7.0,NaN,Ronald Agenor,R,180.0,USA,35.1,104156,NaN,WC,Sunil Kumar,R,NaN,IND,16.7,6-4 6-4,3,R32,82.0,3.0,0.0,64.0,35.0,26.0,19.0,10.0,2.0,3.0,2.0,3.0,62.0,32.0,22.0,16.0,10.0,1.0,4.0,96.0,492.0,1199.0,3.0
69918,2000-891,Chennai,Hard,32,A,20000103,3,210013,NaN,NaN,Martin Damm Sr,R,188.0,CZE,27.4,103294,NaN,NaN,Christophe Rochus,R,170.0,BEL,21.0,6-3 6-0,3,R32,50.0,7.0,0.0,46.0,26.0,23.0,12.0,8.0,1.0,1.0,1.0,1.0,37.0,17.0,8.0,9.0,7.0,1.0,5.0,NaN,NaN,121.0,385.0
69917,2000-891,Chennai,Hard,32,A,20000103,2,102455,NaN,NaN,Lorenzo Manta,R,190.0,SUI,25.2,102720,NaN,NaN,Tomas Zib,R,178.0,CZE,23.9,7-5 6-2,3,R32,80.0,10.0,1.0,69.0,42.0,31.0,16.0,10.0,4.0,4.0,4.0,2.0,70.0,50.0,31.0,9.0,10.0,5.0,8.0,115.0,399.0,111.0,408.0
67588,2000-339,Adelaide,Hard,32,A,20000103,30,103720,6.0,NaN,Lleyton Hewitt,R,180.0,AUS,18.8,102765,8.0,NaN,Nicolas Escude,R,185.0,FRA,23.7,3-6 6-1 6-4,3,SF,115.0,4.0,1.0,82.0,40.0,26.0,27.0,13.0,3.0,5.0,10.0,4.0,81.0,36.0,27.0,19.0,13.0,0.0,4.0,25.0,1429.0,37.0,995.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4103,2024-7694,Lyon,Clay,32,A,20240520,276,208659,NaN,WC,Giovanni Mpetshi Perricard,R,NaN,FRA,20.8,132283,NaN,NaN,Lorenzo Sonego,R,191.0,ITA,29.0,6-3 6-4,3,R32,79.0,8.0,0.0,50.0,34.0,28.0,12.0,10.0,0.0,0.0,2.0,3.0,53.0,37.0,28.0,7.0,9.0,2.0,4.0,117.0,541.0,50.0,991.0
4090,2024-7694,Lyon,Clay,32,A,20240520,291,209260,NaN,NaN,Luciano Darderi,R,NaN,ITA,22.2,105173,3,NaN,Adrian Mannarino,L,183.0,FRA,35.8,7-6(4) 6-3,3,R16,120.0,3.0,0.0,69.0,53.0,40.0,8.0,11.0,2.0,4.0,6.0,0.0,81.0,47.0,33.0,13.0,10.0,9.0,12.0,47.0,1002.0,22.0,1865.0
4088,2024-7694,Lyon,Clay,32,A,20240520,293,136440,NaN,NaN,Dominik Koepfer,L,180.0,GER,30.0,200005,1,NaN,Ugo Humbert,L,188.0,FRA,25.9,2-6 6-4 6-4,3,R16,124.0,6.0,2.0,76.0,54.0,34.0,11.0,14.0,5.0,9.0,2.0,2.0,93.0,62.0,41.0,13.0,14.0,1.0,5.0,67.0,767.0,17.0,2285.0
4102,2024-7694,Lyon,Clay,32,A,20240520,277,106415,NaN,NaN,Yoshihito Nishioka,L,170.0,JPN,28.6,110751,NaN,LL,Sebastian Fanselow,U,NaN,GER,32.3,6-0 6-1,3,R32,62.0,0.0,0.0,37.0,25.0,19.0,11.0,7.0,1.0,1.0,1.0,3.0,49.0,27.0,15.0,6.0,6.0,6.0,11.0,72.0,741.0,316.0,166.0


Already it's clear that there are many columns that are data type "object" for which I will have to turn into numeric, or drop. For many columns I am also seeing a lot of nulls. Let's look into these columns, and if I want to keep them then I will simply drop the null values if there aren't that many.

In [100]:
df.shape

(73247, 49)

In [101]:
df.drop_duplicates(inplace=True)

In [102]:
df.shape # no duplicates

(73247, 49)

In [103]:
# checking for nulls
df.isna().sum(axis=0).sort_values(ascending=False)

winner_entry          64018
loser_entry           58240
loser_seed            56389
winner_seed           42803
minutes                7938
l_1stIn                6469
l_svpt                 6469
l_ace                  6469
l_df                   6469
w_1stWon               6469
w_ace                  6469
w_df                   6469
w_svpt                 6469
w_2ndWon               6469
w_1stIn                6469
l_1stWon               6469
l_2ndWon               6469
l_bpFaced              6469
l_bpSaved              6469
w_bpFaced              6469
w_bpSaved              6469
w_SvGms                6468
l_SvGms                6468
loser_ht               3621
winner_ht              1896
loser_rank             1453
loser_rank_points      1453
winner_rank             566
winner_rank_points      566
surface                  53
loser_hand               42
loser_age                12
winner_age                7
winner_hand               7
winner_ioc                0
loser_id            

In [105]:
missing_stats = df.isna().sum(axis=0).sort_values(ascending=False)/df.shape[0]
missing_stats

winner_entry          0.874002
loser_entry           0.795118
loser_seed            0.769847
winner_seed           0.584365
minutes               0.108373
l_1stIn               0.088318
l_svpt                0.088318
l_ace                 0.088318
l_df                  0.088318
w_1stWon              0.088318
w_ace                 0.088318
w_df                  0.088318
w_svpt                0.088318
w_2ndWon              0.088318
w_1stIn               0.088318
l_1stWon              0.088318
l_2ndWon              0.088318
l_bpFaced             0.088318
l_bpSaved             0.088318
w_bpFaced             0.088318
w_bpSaved             0.088318
w_SvGms               0.088304
l_SvGms               0.088304
loser_ht              0.049435
winner_ht             0.025885
loser_rank            0.019837
loser_rank_points     0.019837
winner_rank           0.007727
winner_rank_points    0.007727
surface               0.000724
loser_hand            0.000573
loser_age             0.000164
winner_a

Winner entry and loser entry are expected to be high. Those columns are there to indicate if a player has come from the qualifications for the tournament or is a wildcard.  This is generally not the case. For now I don't think that those columns are needed, so I am going to go ahead and drop them. I am also going to drop `winner_seed` and `loser_seed` because both those columns are redundant since we have the ranking of the winner and loser too.

In [106]:
df.drop(columns=['winner_entry', 'loser_entry', 'loser_seed', 'winner_seed'], inplace=True)

In [107]:
missing_stats = df.isna().sum(axis=0).sort_values(ascending=False)/df.shape[0]
missing_stats

minutes               0.108373
l_1stIn               0.088318
l_svpt                0.088318
l_1stWon              0.088318
l_df                  0.088318
l_bpFaced             0.088318
w_1stWon              0.088318
w_df                  0.088318
w_ace                 0.088318
w_2ndWon              0.088318
w_1stIn               0.088318
w_svpt                0.088318
l_2ndWon              0.088318
l_bpSaved             0.088318
l_ace                 0.088318
w_bpFaced             0.088318
w_bpSaved             0.088318
l_SvGms               0.088304
w_SvGms               0.088304
loser_ht              0.049435
winner_ht             0.025885
loser_rank_points     0.019837
loser_rank            0.019837
winner_rank_points    0.007727
winner_rank           0.007727
surface               0.000724
loser_hand            0.000573
loser_age             0.000164
winner_hand           0.000096
winner_age            0.000096
winner_ioc            0.000000
winner_id             0.000000
winner_n

The next group of numbers all into stats about the match that was played.  These numbers are not helpful to be worked with because they're happening after the match has already started. For now I am going to drop these numbers, but I may return to them if I want to look at previous match stats as predictors.

In [108]:
df.drop(columns=['minutes','w_ace','w_df','w_svpt','w_1stIn','w_1stWon','w_2ndWon','w_SvGms','w_bpSaved','w_bpFaced','l_ace','l_df','l_svpt','l_1stIn','l_1stWon','l_2ndWon','l_SvGms','l_bpSaved','l_bpFaced'], inplace=True)

In [109]:
missing_stats = df.isna().sum(axis=0).sort_values(ascending=False)/df.shape[0]
missing_stats

loser_ht              0.049435
winner_ht             0.025885
loser_rank            0.019837
loser_rank_points     0.019837
winner_rank           0.007727
winner_rank_points    0.007727
surface               0.000724
loser_hand            0.000573
loser_age             0.000164
winner_hand           0.000096
winner_age            0.000096
winner_id             0.000000
match_num             0.000000
tourney_date          0.000000
draw_size             0.000000
tourney_level         0.000000
tourney_id            0.000000
tourney_name          0.000000
winner_name           0.000000
loser_name            0.000000
winner_ioc            0.000000
loser_id              0.000000
loser_ioc             0.000000
round                 0.000000
score                 0.000000
best_of               0.000000
dtype: float64

For the height columns I'm going to just filter out the null values because they're a small subset of the data.

In [112]:
df.dropna(subset=['loser_ht', 'winner_ht'], inplace=True)

In [113]:
df.isna().sum(axis=0).sort_values(ascending=False)/df.shape[0]

loser_rank            0.003831
loser_rank_points     0.003831
winner_rank           0.001234
winner_rank_points    0.001234
surface               0.000044
loser_hand            0.000029
winner_hand           0.000015
tourney_name          0.000000
tourney_level         0.000000
tourney_id            0.000000
winner_name           0.000000
winner_id             0.000000
match_num             0.000000
tourney_date          0.000000
draw_size             0.000000
winner_ht             0.000000
winner_age            0.000000
winner_ioc            0.000000
loser_ioc             0.000000
loser_ht              0.000000
loser_name            0.000000
loser_id              0.000000
round                 0.000000
best_of               0.000000
score                 0.000000
loser_age             0.000000
dtype: float64

For the next 4 rows I want to completely drop the columns `_rank_points` and just filter the nulls out of the rank columns.

The two columns that I care about for my initial EDA are winner_rank and loser_rank. I want to build my baseline model off of those parameters so any columns where that is null will have to be removed.

In [15]:
df[df['winner_rank'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Index: 566 entries, 69918 to 4206
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          566 non-null    object 
 1   tourney_name        566 non-null    object 
 2   surface             558 non-null    object 
 3   draw_size           566 non-null    int64  
 4   tourney_level       566 non-null    object 
 5   tourney_date        566 non-null    int64  
 6   match_num           566 non-null    int64  
 7   winner_id           566 non-null    int64  
 8   winner_seed         2 non-null      object 
 9   winner_entry        24 non-null     object 
 10  winner_name         566 non-null    object 
 11  winner_hand         559 non-null    object 
 12  winner_ht           161 non-null    float64
 13  winner_ioc          566 non-null    object 
 14  winner_age          559 non-null    float64
 15  loser_id            566 non-null    int64  
 16  loser_se

In [16]:
non_null_fltr1 = df['winner_rank'].notna()
non_null_fltr2 = df['loser_rank'].notna()

# remove those rows, they are a small portion of data
df = df.loc[non_null_fltr1 | non_null_fltr2, :]

In [17]:
df['winner_rank'].value_counts(dropna=False)
df['loser_rank'].value_counts(dropna=False)

# confirmed removed

loser_rank
NaN       1180
44.0       596
46.0       592
39.0       590
37.0       588
          ... 
1545.0       1
1240.0       1
1561.0       1
968.0        1
1576.0       1
Name: count, Length: 1378, dtype: int64

Now in addition to removing the nulls for those columns, I want to restrict the anaysis to only matches that are Grand Slams, and Masters 1000s. These are tournaments that are in the top tier of professional tennis, and I want to exclusively look at those events. If I want to add more tournaments later I can.

Before looking at many of the features I want to do some filtering of the data. The biggest change that I want to make is to only include high level professional tennis tournaments.  These tournaments are called Grand Slams, and Masters 1000s. Thankfully, these tournaments are easy to filter for in the dataset.  I will do that below and then look into other issues.

In [18]:
df['tourney_level'].value_counts()

tourney_level
A    40067
M    13738
G    12192
D     6570
F      407
Name: count, dtype: int64

In [19]:
# update dtype to str


tourney_fltr = ((df['tourney_level'] == 'M') | (df['tourney_level'] == 'G'))

# remove those rows because they're small portion of data
df = df.loc[tourney_fltr, :]

In [20]:
df['tourney_level'].value_counts()

tourney_level
M    13738
G    12192
Name: count, dtype: int64

Now I would like to construct a quick and easy baseline model. For this I just want to see the results if I were to pick the higher ranked player everytime.

In [21]:
higher_rnk_fltr = df['winner_rank'] < df['loser_rank']

df[higher_rnk_fltr].shape[0] / df.shape[0]

0.6749710759737756

This is my baseline model that I will measure the performance of my model against. For that simple model, the winner is correctly picked roughly 68% of the time. 

## Preprocessing and Feature Engineering

For my analysis I don't need a linear regression as I am not predicting a continuous variable. Instead I'm looking to do a binary classification, and so I'm starting with a logistic regression.

In [49]:
from sklearn.linear_model import LogisticRegression

I need to now transform my data into just numerical values for running a logistic regression. What columns are needed?

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25930 entries, 69611 to 4004
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          25930 non-null  object 
 1   tourney_name        25930 non-null  object 
 2   surface             25930 non-null  object 
 3   draw_size           25930 non-null  int64  
 4   tourney_level       25930 non-null  object 
 5   tourney_date        25930 non-null  int64  
 6   match_num           25930 non-null  int64  
 7   winner_id           25930 non-null  int64  
 8   winner_seed         13190 non-null  object 
 9   winner_entry        3032 non-null   object 
 10  winner_name         25930 non-null  object 
 11  winner_hand         25930 non-null  object 
 12  winner_ht           25827 non-null  float64
 13  winner_ioc          25930 non-null  object 
 14  winner_age          25930 non-null  float64
 15  loser_id            25930 non-null  int64  
 16  loser_

In [24]:
df_final = df.copy()

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25930 entries, 69611 to 4004
Data columns (total 49 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tourney_id          25930 non-null  object 
 1   tourney_name        25930 non-null  object 
 2   surface             25930 non-null  object 
 3   draw_size           25930 non-null  int64  
 4   tourney_level       25930 non-null  object 
 5   tourney_date        25930 non-null  int64  
 6   match_num           25930 non-null  int64  
 7   winner_id           25930 non-null  int64  
 8   winner_seed         13190 non-null  object 
 9   winner_entry        3032 non-null   object 
 10  winner_name         25930 non-null  object 
 11  winner_hand         25930 non-null  object 
 12  winner_ht           25827 non-null  float64
 13  winner_ioc          25930 non-null  object 
 14  winner_age          25930 non-null  float64
 15  loser_id            25930 non-null  int64  
 16  loser_

In [52]:
df_final = df_final.select_dtypes('number')

df_final

,draw_size,tourney_date,match_num,winner_id,winner_ht,winner_age,loser_id,loser_ht,loser_age,best_of,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
69611,128,20000117,71,102271,175.0,26.2,102966,193.0,22.8,5,107.0,11.0,5.0,103.0,63.0,54.0,17.0,14.0,6.0,6.0,3.0,0.0,100.0,57.0,42.0,19.0,15.0,8.0,11.0,41.0,800.0,123.0,313.0
69629,128,20000117,89,103096,173.0,22.0,102839,188.0,23.4,5,135.0,5.0,1.0,104.0,62.0,48.0,20.0,17.0,1.0,4.0,11.0,6.0,114.0,73.0,53.0,16.0,17.0,10.0,14.0,54.0,694.0,7.0,1975.0
69594,128,20000117,54,102133,180.0,27.1,103252,175.0,21.4,5,148.0,16.0,6.0,148.0,76.0,56.0,43.0,21.0,10.0,12.0,5.0,6.0,114.0,76.0,52.0,24.0,20.0,2.0,7.0,89.0,439.0,52.0,715.0
69595,128,20000117,55,101962,193.0,28.3,103157,193.0,21.8,5,175.0,14.0,4.0,172.0,103.0,68.0,38.0,23.0,14.0,19.0,10.0,4.0,145.0,91.0,68.0,26.0,24.0,3.0,10.0,29.0,991.0,257.0,124.0
69596,128,20000117,56,103163,188.0,21.7,102642,190.0,24.4,5,144.0,17.0,7.0,132.0,65.0,58.0,34.0,20.0,6.0,7.0,16.0,3.0,113.0,65.0,50.0,25.0,20.0,5.0,8.0,12.0,1510.0,171.0,215.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4000,128,20240506,259,208859,NaN,23.3,134868,188.0,27.8,3,142.0,7.0,6.0,95.0,56.0,45.0,24.0,16.0,2.0,3.0,9.0,4.0,94.0,57.0,50.0,15.0,15.0,6.0,7.0,240.0,248.0,26.0,1595.0
4017,128,20240506,242,208029,185.0,21.0,208134,185.0,20.7,3,102.0,1.0,1.0,67.0,39.0,31.0,17.0,10.0,6.0,6.0,1.0,3.0,59.0,28.0,20.0,17.0,10.0,2.0,4.0,12.0,3250.0,81.0,718.0
4002,128,20240506,257,126203,193.0,26.5,104926,178.0,36.9,3,79.0,6.0,1.0,62.0,35.0,29.0,16.0,10.0,2.0,2.0,3.0,4.0,51.0,30.0,23.0,10.0,9.0,1.0,3.0,13.0,2870.0,92.0,650.0
4003,128,20240506,256,200624,196.0,23.8,207925,NaN,22.0,3,163.0,4.0,0.0,101.0,50.0,39.0,30.0,16.0,3.0,5.0,7.0,4.0,120.0,65.0,48.0,29.0,16.0,7.0,9.0,27.0,1525.0,57.0,868.0


In [53]:
df_final.isna().sum(axis=0).sort_values(ascending=False)

minutes               790
loser_ht              205
l_bpFaced             197
w_1stIn               197
w_2ndWon              197
w_svpt                197
w_df                  197
w_ace                 197
w_1stWon              197
l_2ndWon              197
l_1stWon              197
l_svpt                197
l_1stIn               197
l_df                  197
l_ace                 197
w_bpFaced             197
w_bpSaved             197
l_bpSaved             197
w_SvGms               196
l_SvGms               196
winner_ht             103
loser_rank             36
loser_rank_points      36
winner_rank             9
winner_rank_points      9
draw_size               0
best_of                 0
match_num               0
winner_age              0
winner_id               0
loser_age               0
loser_id                0
tourney_date            0
dtype: int64

Let's work on creating my target variable first

### Presentation Code Snippets

#### Preprocessing Slide

In [42]:
some_data = np.random.randint(low=0,high=100,size=(1000,11))

In [43]:
mock_df_raw = pd.DataFrame(some_data,
                  index=list("match_{}".format(i) for i in range(some_data.shape[0])),
                  columns= list("WIN_feat{}".format(i) for i in range(5)) + 
                           list("LOS_feat{}".format(i) for i in range(5))  + ["NULL"])

mock_df_raw

,WIN_feat0,WIN_feat1,WIN_feat2,WIN_feat3,WIN_feat4,LOS_feat0,LOS_feat1,LOS_feat2,LOS_feat3,LOS_feat4,NULL
match_0,52,6,0,96,14,36,19,27,0,68,1
match_1,25,52,49,62,39,19,43,15,17,99,54
match_2,66,6,44,78,13,15,77,71,7,80,13
match_3,2,96,50,32,46,33,34,51,69,16,58
match_4,40,63,49,9,66,10,66,69,80,46,49
...,...,...,...,...,...,...,...,...,...,...,...
match_995,39,34,99,72,82,31,40,18,59,1,19
match_996,40,66,77,32,87,32,7,33,50,86,10
match_997,14,38,78,67,46,99,50,38,50,14,15
match_998,15,17,78,50,35,46,87,79,19,88,78


In [44]:
formatted_mock_df = pd.DataFrame(some_data, 
                      columns=list("HR_feat{}".format(i) for i in range(5)) + 
                      list("LR_feat{}".format(i) for i in range(5)) + ["HR_win"],
                      index=mock_df.index)
formatted_mock_df['HR_win'] = np.where(formatted_mock_df['HR_win'] > 50, 1, 0)

formatted_mock_df

,HR_feat0,HR_feat1,HR_feat2,HR_feat3,HR_feat4,LR_feat0,LR_feat1,LR_feat2,LR_feat3,LR_feat4,HR_win
match_0,52,6,0,96,14,36,19,27,0,68,0
match_1,25,52,49,62,39,19,43,15,17,99,1
match_2,66,6,44,78,13,15,77,71,7,80,0
match_3,2,96,50,32,46,33,34,51,69,16,1
match_4,40,63,49,9,66,10,66,69,80,46,0
...,...,...,...,...,...,...,...,...,...,...,...
match_995,39,34,99,72,82,31,40,18,59,1,0
match_996,40,66,77,32,87,32,7,33,50,86,0
match_997,14,38,78,67,46,99,50,38,50,14,0
match_998,15,17,78,50,35,46,87,79,19,88,1


In [73]:
pd.set_option('display.max_columns', 100)
df_raw

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,Kei Nishikori,R,178.0,JPN,29.0,106421,4.0,NaN,Daniil Medvedev,R,198.0,RUS,22.8,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4.0,NaN,Daniil Medvedev,R,198.0,RUS,22.8,104542,NaN,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.7,7-6(6) 6-2,3,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2.0,NaN,Kei Nishikori,R,178.0,JPN,29.0,104871,NaN,NaN,Jeremy Chardy,R,188.0,FRA,31.8,6-2 6-2,3,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NaN,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.7,200282,7.0,NaN,Alex De Minaur,R,183.0,AUS,19.8,6-4 7-6(2),3,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4.0,NaN,Daniil Medvedev,R,198.0,RUS,22.8,105683,5.0,NaN,Milos Raonic,R,196.0,CAN,28.0,6-7(2) 6-3 6-4,3,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73242,2014-605,Tour Finals,Hard,8,F,20141109,514,104925,1.0,NaN,Novak Djokovic,R,188.0,SRB,27.4,105453,4.0,NaN,Kei Nishikori,R,178.0,JPN,24.8,6-1 3-6 6-0,3,SF,88.0,3.0,2.0,58.0,34.0,28.0,13.0,11.0,2.0,4.0,3.0,6.0,67.0,38.0,29.0,5.0,11.0,3.0,9.0,1.0,10010.0,5.0,4625.0
73243,2014-605,Tour Finals,Hard,8,F,20141109,515,104925,1.0,NaN,Novak Djokovic,R,188.0,SRB,27.4,103819,2.0,NaN,Roger Federer,R,185.0,SUI,33.2,W/O,3,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10010.0,2.0,8700.0
73244,2014-D015,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,1,104527,NaN,NaN,Stan Wawrinka,R,183.0,SUI,29.6,104542,NaN,NaN,Jo-Wilfried Tsonga,R,188.0,FRA,29.5,6-1 3-6 6-3 6-2,5,RR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5295.0,12.0,2740.0
73245,2014-D015,Davis Cup WG F: FRA vs SUI,Clay,4,D,20141121,2,104792,NaN,NaN,Gael Monfils,R,193.0,FRA,28.2,103819,NaN,NaN,Roger Federer,R,185.0,SUI,33.2,6-1 6-4 6-3,5,RR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,1825.0,2.0,9700.0


In [ ]:
df_final